## The Deepnote Notebook
The most important part of Deepnote is the notebook. This is where the magic happens: it's where you read, analyze, and visualize your data.

In [1]:
import os
import io
import random
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import default_loader
from torch.utils.data import RandomSampler
import torchvision.transforms as tt
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:



DATA_DIR = "/work/media"
sample_dir = '/work/output'

imageSize = 256
batchSize = 64
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
latentSize = 256
imageRows = 8
imageCols = 8
featureSize = 64

imageTransforms = tt.Compose([ tt.Resize(imageSize),
                               tt.CenterCrop(imageSize),
                               tt.ToTensor()])


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

fixedLatent = torch.randn(imageRows * imageCols, latentSize, 1, 1, device=device)

trainDataset = ImageFolder(DATA_DIR, transform=imageTransforms)

trainDataLoader = DataLoader(trainDataset, batchSize, shuffle=True, num_workers=2, pin_memory=True)



In [3]:
def getRandomImagePath():
    """
    pulls random image from dataset
    """
    randFolder = random.choice(os.listdir(DATA_DIR))
    randImage = random.choice(os.listdir(DATA_DIR + '/' + randFolder))
    return DATA_DIR + '/' + randFolder + '/' + randImage

In [ ]:
generator = nn.Sequential(
            # input is latentSize x 1 x 1
              nn.ConvTranspose2d(latentSize, featureSize * 32, 4, 1, 0, bias=False),
              nn.BatchNorm2d(featureSize * 32),
              nn.ReLU(True),
              # state size. (featureSize*32) x 4 x 4
              nn.ConvTranspose2d(featureSize * 32, featureSize * 16, 4, 2, 0, bias=False),
              nn.BatchNorm2d(featureSize * 16),
              nn.ReLU(True),
              # state size. (featureSize*16) x 8 x 8
              nn.ConvTranspose2d(featureSize * 16, featureSize * 8, 4, 2, 1, bias=False),
              nn.BatchNorm2d(featureSize * 8),
              nn.ReLU(True),
              # state size. (featureSize*8) x 16 x 16
              nn.ConvTranspose2d( featureSize * 8, featureSize * 4, 4, 2, 1, bias=False),
              nn.BatchNorm2d(featureSize * 4),
              nn.ReLU(True),
              # state size. (featureSize*4) x 32 x 32
              nn.ConvTranspose2d( featureSize * 4, featureSize * 2, 4, 2, 1, bias=False),
              nn.BatchNorm2d(featureSize * 2),
              nn.ReLU(True),
              # state size. (featureSize*2) x 64 x 64
              nn.ConvTranspose2d( featureSize * 2, featureSize, 4, 2, 1, bias=False),
              nn.BatchNorm2d(featureSize),
              nn.ReLU(True),
              # state size. (featureSize) x 128 x 128
              nn.ConvTranspose2d(featureSize, 3, 4, 2, 1, bias=False),
              nn.Tanh()
              # state size. 3 x 256 x 256
            )


    

def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

def saveSamples(index, latent_tensors, imageRows, imageCols, show=True):
    fake_images = generator(latent_tensors)
    fake_fname = sample_dir +'/' + 'generated-images-{0:0=4d}.png'.format(index)
    print(fake_images.size())
    save_image(denorm(fake_images), fake_fname, nrow=imageRows)

def merge(img1, img2, w, h, transparency):
    """
    Superimpose image 2 on top of image 1 with a transparency filter.
    Both img1 and img2 need to be in the form of a PIL image.
    Also resizes both images to specified width and height.
    Returns a modified img1.
    """
    # resize the image
    size = (w,h)
    img2 = img2.resize(size,Image.ANTIALIAS)
    img1 = img1.resize(size,Image.ANTIALIAS)
    im = Image.blend(img2, img1, transparency)
    return im

from google.cloud import vision
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'hackalytics-2022-edddba28c1d0.json'
client = vision.ImageAnnotatorClient()

def faceDiscriminator(image):
    """returns the landmarking confience of Vision as float for the first face detected.
    Returns 0 if Vision cannot detect a face."""
    with io.open(image, 'rb') as image_file:
        content = image_file.read()
    vision_image = vision.Image(content=content)

    response = client.face_detection(image=vision_image)
    faces = response.face_annotations
    for face in faces:
        return face.landmarking_confidence
    return 0



def discriminator(images):
    """
    Vision determines if a series of images can be
    landmarked or not.
    Takes in a tensor of images.
    Returns a tensor of the shape (images.size(0), 1)
    """
    outTensor = torch.zeros(images.size(0), 1)
    images = torch.clone(images).detach()
    for idx, image in enumerate(images):
        img = torch.transpose(image, 0, 2)
        img = img.numpy()
        img = Image.fromarray(img, mode="RGB")
        baseImg = Image.open(getRandomImagePath())
        merge(img, baseImg, imageSize, imageSize, 0.95).save("temp.png")
        outTensor[idx] = faceDiscriminator("temp.png")
    return outTensor.requires_grad_()

def assessDiscriminator():
    latent = torch.randn(batchSize, latentSize, 1, 1, device=device)
    fake_images = generator(latent)
    fake_targets = torch.ones(fake_images.size(0), 1, device=device)
    fake_preds = discriminator(fake_images)
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)
    score = torch.mean(fake_preds).item()
    return score
    

def train_generator(opt_g):
    # Clear generator gradients
    opt_g.zero_grad()
    
    # Generate fake images
    latent = torch.randn(batchSize, latentSize, 1, 1, device=device)
    fake_images = generator(latent)
    
    # Try to fool the discriminator
    preds = discriminator(fake_images)
    targets = torch.ones(batchSize, 1, device=device)
    loss = F.binary_cross_entropy(preds, targets)
    
    # Update generator weights
    loss.backward()
    opt_g.step()
    
    return loss.item()


def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    scores = []
    
    # Create optimizers
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(1, epochs + 1):
        for real_images, _ in tqdm(trainDataLoader):
            # Train generator
            score = assessDiscriminator();
            loss_g = train_generator(opt_g)
            
        # Record losses & scores
        losses_g.append(loss_g)
        scores.append(score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, score: {:.4f}".format(
            epoch+1, epochs, loss_g, score))
        # Save model weights
        generatorWeights = torch.save(generator, sample_dir + '/' + 
                                      str(epoch) + 'generator.pth')    

        # Save generated images
        saveSamples(epoch+start_idx, fixedLatent, imageRows, imageCols, show=False)


    
    return losses_g, scores



if __name__ == '__main__':
    lr = 0.0002
    epochs = 200
    history = fit(epochs, lr)
    losses_g, scores = history

  0%|          | 0/11 [00:00<?, ?it/s]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d80e1763-e9c4-41f1-9d3f-7a71b565f358' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>